In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
import os
import cv2
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


In [4]:
import os

output_real_dir = "/content/drive/MyDrive/Output/working/frames/real"
output_spoof_dir = "/content/drive/MyDrive/Output/working/frames/spoof"

os.makedirs(output_real_dir, exist_ok=True)
os.makedirs(output_spoof_dir, exist_ok=True)


In [5]:
real_videos_dir = "/content/drive/MyDrive/Video classification/Data sets/Real"
spoof_videos_dir = "/content/drive/MyDrive/Video classification/Data sets/Fake"

output_real_dir = "/content/drive/MyDrive/Output/working/frames/real"
output_spoof_dir = "/content/drive/MyDrive/Output/working/frames/spoof"

os.makedirs(output_real_dir, exist_ok=True)
os.makedirs(output_spoof_dir, exist_ok=True)


In [6]:
import cv2
import os

def check_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps if fps else 0
    print(f"Video: {video_path}\nFrames: {total_frames}, FPS: {fps}, Duration: {duration:.2f}s\n")
    cap.release()

# Check all spoof videos
for f in os.listdir(spoof_videos_dir):
    if f.endswith(('.mp4', '.avi', '.mov')):
        check_video_frames(os.path.join(spoof_videos_dir, f))


Video: /content/drive/MyDrive/Video classification/Data sets/Fake/Fake3.mp4
Frames: 1044, FPS: 24.0, Duration: 43.50s



In [7]:
def extract_frames_from_videos(videos_dir, output_dir, label, max_videos=5):
    video_files = [f for f in os.listdir(videos_dir) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    video_files = video_files[:max_videos]

    for video_file in video_files:
        video_path = os.path.join(videos_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        success, image = cap.read()

        while success:
            if frame_count % int(cap.get(cv2.CAP_PROP_FPS)) == 0:
                frame_filename = f"{label}_{video_file}_frame{frame_count // int(cap.get(cv2.CAP_PROP_FPS))}.jpg"
                frame_path = os.path.join(output_dir, frame_filename)
                cv2.imwrite(frame_path, image)
            success, image = cap.read()
            frame_count += 1

        cap.release()

if not os.listdir(output_real_dir):
    extract_frames_from_videos(real_videos_dir, output_real_dir, "real")
if not os.listdir(output_spoof_dir):
    extract_frames_from_videos(spoof_videos_dir, output_spoof_dir, "spoof")

print("Frame extraction completed.")


Frame extraction completed.


In [8]:
print("Real frames:", len(os.listdir(output_real_dir)))
print("Spoof frames:", len(os.listdir(output_spoof_dir)))


Real frames: 160
Spoof frames: 147


In [9]:
import shutil
import os

# Create the new path if it doesn't exist
if not os.path.exists("/content/frames"):
    os.makedirs("/content/frames")

# Copy real and spoof folders
shutil.copytree("/content/drive/MyDrive/Output/working/frames/real", "/content/frames/real", dirs_exist_ok=True)
shutil.copytree("/content/drive/MyDrive/Output/working/frames/spoof", "/content/frames/spoof", dirs_exist_ok=True)

print("Copied folders to /content/frames")


Copied folders to /content/frames


In [10]:
# Define transform (you already fixed this)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ NOW define the dataset BEFORE splitting it
dataset_dir = "/content/frames"
dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)

# ✅ THEN do the splitting
small_dataset, _ = torch.utils.data.random_split(dataset, [200, len(dataset) - 200])
train_dataset, val_dataset = torch.utils.data.random_split(small_dataset, [160, 40])


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('resnet18', pretrained=True, num_classes=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [12]:
num_epochs = 1
best_val_accuracy = 0
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%")

    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_vit_model.pth')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break

    scheduler.step()
print(f"Best Validation Accuracy: {best_val_accuracy:.2f}%")


Epoch [1/1], Loss: 0.7002, Training Accuracy: 48.12%
Validation Accuracy: 50.00%
Best Validation Accuracy: 50.00%


In [13]:
def predict_video(video_path, model, transform, device):
    cap = cv2.VideoCapture(video_path)
    real_count = 0
    spoof_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)

        if predicted.item() == 0:
            real_count += 1
        else:
            spoof_count += 1

    cap.release()

    if real_count > spoof_count:
        print(f"Result: Real video ({real_count} real frames, {spoof_count} spoof frames)")
        return "Real"
    else:
        print(f"Result: Spoof video ({real_count} real frames, {spoof_count} spoof frames)")
        return "Spoof"


In [17]:
# Use the correct architecture (ResNet18) that matches your saved model
model = timm.create_model('resnet18', pretrained=False, num_classes=2)

# Load the weights that were saved from a ResNet18 model
model.load_state_dict(torch.load('best_vit_model.pth'))

# Move to device and set to eval mode
model.to(device)
model.eval()


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

video_path = "/content/drive/MyDrive/Video classification/Data sets/Fake/Fake3.mp4"
result = predict_video(video_path, model, transform, device)


Result: Spoof video (84 real frames, 960 spoof frames)


In [ ]:
import torch

# Assuming your model is called MyModel (or adjust as needed)
# and you have already trained it

# Set to eval mode before saving (recommended)
model.eval()

# Save only the weights
torch.save(model.state_dict(), 'model.pth')

print("✅ Model saved as model.pth")



✅ Model saved as model.pth
